In [ ]:
# Quelle est l'influence combinée des facteurs démographiques, économiques, de sécurité et sociaux sur les résultats des élections présidentielles de 2022 en France, par code postal?

In [ ]:
# Feature Engineering

# Ajout d'une tranche d'age dans la démographie
# Faire le ratio entre le niveau de vie et le niveau de chomage
# Ajouter le taux de criminalité

In [ ]:
# Ajout d'une tranche d'age dans la démographie

In [2]:
from models.managers.db_manager import connect_to_bdd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.model_selection import GridSearchCV


# Création des requêtes SQL et des dataframes
conn = connect_to_bdd()

query_demo = "SELECT * FROM demographie"
df_demo = pd.read_sql(query_demo, conn)

query_economie = "SELECT * FROM economie"
df_economie = pd.read_sql(query_economie, conn)

query_securite = "SELECT * FROM securite"
df_securite = pd.read_sql(query_securite, conn)

query_social = "SELECT * FROM social"
df_social = pd.read_sql(query_social, conn)

query_election_t1 = "SELECT * FROM election_2022_t1"
df_election_t1 = pd.read_sql(query_election_t1, conn)

# Fermer la connexion
conn.close()

# Fusionner les dataframes sur 'code_postal'
df_merged = df_demo.merge(df_economie, on='code_postal', how='inner')
df_merged = df_merged.merge(df_securite, on='code_postal', how='inner')
df_merged = df_merged.merge(df_social, on='code_postal', how='inner')


/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_75632/1931027635.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)
/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_75632/1931027635.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_75632/1931027635.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_securite = pd.read_sql(query_securite, conn)
/var/fo

In [ ]:
# Ajout d'une tranche d'age dans la démographie (pas sur que sa fonctionne)
df_merged['jeunes_adultes'] = df_merged['0-24_2023'] + df_merged['25-59_2023']
df_merged['seniors'] = df_merged['60-74_2023'] + df_merged['75+_2023']

In [ ]:
# Calculer le ratio entre le niveau de vie médian et le taux de chômage (simplification hypothétique)
df_merged['ratio_vie_chomage'] = df_merged['Médiane du niveau de vie 2021'] / df_merged['avg_2022']

# Ajouter le taux de criminalité (supposons que 'tauxpourcent' représente cela)
df_merged['taux_criminalite'] = df_merged['tauxpourcent']

# Afficher les premières lignes du dataframe
print(df_merged.head())

In [ ]:
# Préparation des données

X = df_merged.drop(['parti_gagnant', 'code_postal'], axis=1)  # Supprimez 'code_postal' s'il ne sert pas de feature
y = df_merged['parti_gagnant']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Configuration du modèle XGBoost
# Entrainement du model
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique()), seed=42)

# Entraînement du modèle
model.fit(X_train, y_train)

In [ ]:
# prédiction
y_pred = model.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Rapport de classification et matrice de confusion
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Importance des features pour comprendre les résultats, et qu'elles variables influent le plus sur les résultats

# Pourrais être assimilé à une phase de visualisation

importance = model.feature_importances_
features = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print(features)


In [ ]:
# Test de GridSearchCV pour trouver les meilleurs hyperparamètres, histoire d'améliorer les performances du modèle

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [100, 200],
    'objective': ['multi:softmax']
}


In [ ]:
# Configuration du model

# Créer le modèle XGBoost
xgb_model = xgb.XGBClassifier()

# Configurer la recherche de grille avec validation croisée
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2)

# Exécuter la recherche de grille
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres

print("Meilleurs paramètres: ", grid_search.best_params_)
print("Meilleure précision: {:.2f}%".format(grid_search.best_score_ * 100))

In [ ]:
# Utiliser le meilleur modèle pour faire des prédictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluer le meilleur modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du meilleur modèle: {:.2f}%".format(accuracy * 100))
